In [ ]:
!pip3 install gym
!pip3 install gym-anytrading
!pip3 install tensortrade
!pip3 install keras

     |████████████████████████████████| 174kB 4.3MB/s 
     |████████████████████████████████| 32.6MB 126kB/s 
     |████████████████████████████████| 645kB 31.0MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 788kB 33.1MB/s 
     |████████████████████████████████| 13.2MB 37.1MB/s 
     |████████████████████████████████| 368kB 34.1MB/s 
  Created wheel for tensortrade: filename=tensortrade-1.0.3-cp37-none-any.whl size=134872 sha256=a0b6bfcae6219ef11c0627d5b92eb48fac9f0b85894604868f408d46b2e9db71
  Stored in directory: /root/.cache/pip/wheels/c2/17/ec/2a59434ddc304c85c69d61fadcafdc1d0f44cc847255abf78f
Successfully built tensortrade
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.24.1 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling Py

In [ ]:
import gym
import numpy as np
import gym_anytrading
import pandas as pd
import random
import tensorflow as tf 
from tensortrade.data.cdd import CryptoDataDownload
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from keras.optimizers import SGD
import os 
output_dir = 'model_output/cartpole/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

window_size = 10
frame_bound = (10, len(df))
customenv = gym.make('stocks-v0',df = df,window_size = 10,frame_bound = (10, len(df)))
state_size = customenv.observation_space.shape[0]
action_size = customenv.action_space.n
batch_size = 50
n_episodes = 1000
state_size

customenv.shape[1]







2

DQN Agent

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.hidden_size = state_size*2
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 0.9 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.05 # minimum amount of random exploration permitted
        self.learning_rate = 0.0001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    def _build_model(self):
      hidden_size = state_size*2
      model = Sequential()
      model.add(Dense(hidden_size, input_shape=[customenv.shape[1]], activation='relu'))
      model.add(Dense(hidden_size, activation='relu'))
      model.add(Dense(action_size))
      model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
      
     
      return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        state=np.reshape(state,[10,2])    
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)
    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            state = np.reshape(state,[10,2])
            next_state = np.reshape(next_state,[10,2])
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    def save(self, name):
        self.model.save_weights(name)
    def load(self, name):
        self.model.load_weights(name)



In [ ]:
agent = DQNAgent(state_size, action_size) 


In [ ]:

done = False

for e in range(n_episodes): # iterate over new episodes of the game
    state = customenv.reset() # reset state at start of each new episode of the game
    state = np.reshape(state,[10,2])
    
           
    for time in range(500):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
        #customenv.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = customenv.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        reward = reward if not done else -10 # reward +1 for each additional frame with pole upright  
        next_state = np.reshape(next_state,[10,2])      
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        print("episode: {}/{}, score: {}, e: {}".format(e, n_episodes,customenv.step(action), agent.epsilon))
        #break # exit loop
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
         agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")
              

            






  

Streaming output truncated to the last 5000 lines.
episode: 623/1000, score: (array([[ 2.49434433e+02, -4.80481000e+00],
       [ 2.47462463e+02, -1.97197000e+00],
       [ 2.37742737e+02, -9.71972600e+00],
       [ 2.36261261e+02, -1.48147600e+00],
       [ 2.38818817e+02,  2.55755600e+00],
       [ 2.38773773e+02, -4.50440000e-02],
       [ 2.37972977e+02, -8.00796000e-01],
       [ 2.45475479e+02,  7.50250200e+00],
       [ 2.43058060e+02, -2.41741900e+00],
       [ 2.41426422e+02, -1.63163800e+00]]), 0, False, {'total_reward': -9.564559000000088, 'total_profit': 0.7898981413243789, 'position': 0}), e: 0.04990684317543711
episode: 623/1000, score: (array([[ 2.37742737e+02, -9.71972600e+00],
       [ 2.36261261e+02, -1.48147600e+00],
       [ 2.38818817e+02,  2.55755600e+00],
       [ 2.38773773e+02, -4.50440000e-02],
       [ 2.37972977e+02, -8.00796000e-01],
       [ 2.45475479e+02,  7.50250200e+00],
       [ 2.43058060e+02, -2.41741900e+00],
       [ 2.41426422e+02, -1.63163800e+0

KeyboardInterrupt: ignored